In [1]:
import itertools
import os
import json
import copy
import sys
from collections import defaultdict
import itertools
import pandas as pd
from tqdm.notebook import tqdm
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
import seaborn as sns
import scipy.stats as stats
import pandas as pd
#import seaborn as sns
import seaborn as sns
from matplotlib import rcParams
import scipy
from scipy import stats
import seaborn as sns
from scipy import stats
import sys
import seaborn as sns
from utils import *

In [2]:
AVAILBLE_DATASETS = ['DIALOG_pc.csv','DIALOG_tc.csv','FLICKR.csv','MLQE.csv','REAL_SUM.csv',
                     'SUM_EVAL.csv','TAC_08.csv','TAC_09.csv','TAC_11.csv']
data_path = 'final_df'
ds_to_load = 'DIALOG_pc.csv'

In [3]:
count = 0
for ds in AVAILBLE_DATASETS:
    print('--------------------------------------')
    print( 'Loading DS',ds)
    considered_df = pd.read_csv(os.path.join(data_path,ds)).set_index(['System','Utterance'])
    print(considered_df.shape)
    count += considered_df.shape[0]*considered_df.shape[1]
    is_metric = ['H:' in i for i in considered_df.columns]
    human_metrics = []
    for index,i in enumerate(considered_df.columns):
        if is_metric[index]: 
            print('Human',i)
            human_metrics.append(i)
        else:
            pass
            print('Metric',i)
print('Number of points',count)

--------------------------------------
Loading DS DIALOG_pc.csv
(240, 19)
Human H:engaging
Human H:context
Human H:natural
Human H:overall
Human H:understandable
Human H:useknowledge
Metric S3_pyr
Metric S3_resp
Metric ROUGE_WE_1
Metric ROUGE_WE_2
Metric JS_1
Metric JS_2
Metric ROUGE_L
Metric ROUGE_1
Metric ROUGE_2
Metric BLEU
Metric Chrfpp
Metric BERTScore
Metric MoverScore
--------------------------------------
Loading DS DIALOG_tc.csv
(300, 19)
Human H:engaging
Human H:context
Human H:natural
Human H:overall
Human H:understandable
Human H:useknowledge
Metric S3_pyr
Metric S3_resp
Metric ROUGE_WE_1
Metric ROUGE_WE_2
Metric JS_1
Metric JS_2
Metric ROUGE_L
Metric ROUGE_1
Metric ROUGE_2
Metric BLEU
Metric Chrfpp
Metric BERTScore
Metric MoverScore
--------------------------------------
Loading DS FLICKR.csv
(864, 14)
Human H:overall
Metric S3_pyr
Metric S3_resp
Metric ROUGE_WE_1
Metric ROUGE_WE_2
Metric JS_1
Metric JS_2
Metric ROUGE_L
Metric ROUGE_1
Metric ROUGE_2
Metric BLEU
Metric Chrf

In [5]:
considered_df.head()

H:pyr  H:resp    S3_pyr   S3_resp  ROUGE_WE_1  ROUGE_WE_2  \
System Utterance                                                              
0      M0         0.675     1.0  0.723394  0.798821    0.555831    0.393484   
       M1         1.062     1.0  0.740880  0.802141    0.578164    0.398496   
       M2         1.030     1.0  0.735437  0.795142    0.560794    0.393484   
       M3         0.763     1.0  0.678842  0.753617    0.533499    0.385965   
       M4         0.553     0.8  0.277277  0.402272    0.347395    0.155388   

                      JS_1      JS_2   ROUGE_L   ROUGE_1   ROUGE_2  \
System Utterance                                                     
0      M0        -0.320665 -0.465778  0.401985  0.580645  0.333333   
       M1        -0.301063 -0.465248  0.399504  0.593052  0.325815   
       M2        -0.304503 -0.478472  0.401985  0.588089  0.320802   
       M3        -0.343083 -0.475570  0.372208  0.553350  0.310777   
       M4        -0.469595 -0.637978  0.141439  0.354839  0.067669   

                        BLEU    Chrfpp  BERTScore  MoverScore  
System Utterance                                               
0      M0         100.000000  0.421909   0.726998    0.413697  
       M1         100.000000  1.000000   0.741266    0.444839  
       M2         100.000000  0.468354   0.731971    0.431084  
       M3         100.000000  0.391787   0.720984    0.401022  
       M4           6.190456  0.364575   0.588147    0.156335